In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras import optimizers, metrics

d:\programs\python installed\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
d:\programs\python installed\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
Using TensorFlow backend.
d:\programs\python installed\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [15]:
train_dataX = np.load('augmented_data2.npy')
test_dataX = np.load('test_data.npy')
train_datay = pd.read_csv('Augmented_labels2.csv')

train_datay = train_datay.drop('Id',1)
Y_train = np_utils.to_categorical(train_datay,2)

#splitting data for validation
X_train, X_test, y_train, y_test = train_test_split(train_dataX,Y_train, test_size=0.25, shuffle=True)

In [17]:
del model
model = Sequential()
model.add(Dense(800, input_shape=(44100,), activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 800)               35280800  
_________________________________________________________________
dense_18 (Dense)             (None, 400)               320400    
_________________________________________________________________
dropout_9 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 200)               80200     
_________________________________________________________________
dropout_10 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 402       
Total params: 35,681,802
Trainable params: 35,681,802
Non-trainable params: 0
________________________________________________________________

In [18]:
# compile model
sgd=optimizers.SGD(lr=.1)
adam = optimizers.Adam(lr=0.01)
model.compile(optimizer=adam,
              loss="categorical_crossentropy",
              metrics=["accuracy"])

earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

In [27]:
model.fit(X_train, y_train,
          batch_size=100, nb_epoch=30,verbose=1,
          validation_data=(X_test, y_test))

Train on 4097 samples, validate on 1366 samples
Epoch 1/30
 100/4097 [..............................] - ETA: 4s - loss: 0.6484 - acc: 0.7500

d:\programs\python installed\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


4097/4097 [==============================] - 6s 1ms/step - loss: 0.6873 - acc: 0.7064 - val_loss: 0.7172 - val_acc: 0.6501
Epoch 2/30
4097/4097 [==============================] - 6s 1ms/step - loss: 0.6980 - acc: 0.6895 - val_loss: 0.6919 - val_acc: 0.6537
Epoch 3/30
4097/4097 [==============================] - 5s 1ms/step - loss: 0.6325 - acc: 0.6937 - val_loss: 0.6642 - val_acc: 0.6457
Epoch 4/30
4097/4097 [==============================] - 5s 1ms/step - loss: 0.6316 - acc: 0.6907 - val_loss: 0.6918 - val_acc: 0.6369
Epoch 5/30
4097/4097 [==============================] - 5s 1ms/step - loss: 0.6960 - acc: 0.6788 - val_loss: 0.6323 - val_acc: 0.6413
Epoch 6/30
4097/4097 [==============================] - 4s 1ms/step - loss: 0.6196 - acc: 0.6722 - val_loss: 0.6924 - val_acc: 0.6391
Epoch 7/30
4097/4097 [==============================] - 4s 1000us/step - loss: 0.6234 - acc: 0.6671 - val_loss: 0.7606 - val_acc: 0.6428
Epoch 8/30
4097/4097 [==============================] - 4s 994us/step 

In [28]:
y_prob = model.predict(X_test, verbose=0)
print(y_prob)

roc = roc_auc_score(y_test, y_prob)
print("ROC:" + str(round(roc,2)))

[[0.6191877  0.38081232]
 [0.6191877  0.38081232]
 [1.         0.        ]
 ...
 [0.6191877  0.38081232]
 [0.6191877  0.38081232]
 [0.6191877  0.38081232]]
ROC:0.55


In [ ]:
pd.DataFrame((y_prob[:,1])).to_csv('submissio16.csv', header=["Label"], index_label="Id")